In [1]:
import sys
from pathlib import Path

project_root = Path().absolute().parent
sys.path.append(str(project_root))

In [41]:
from qdrant_client import QdrantClient, models
from rich import print

DENSE_MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
SPARSE_MODEL = "Qdrant/bm25"
collection_name = "episodes_enhanced"
qdrant_client = QdrantClient("http://localhost:6333")
# qdrant_client.set_model(DENSE_MODEL)
# qdrant_client.set_sparse_model(SPARSE_MODEL)

In [42]:
from fastembed.embedding import TextEmbedding
from fastembed.sparse.bm25 import Bm25

dense_embedding_model = TextEmbedding(DENSE_MODEL)
sparse_embedding_model = Bm25(SPARSE_MODEL)

In [126]:
query_text = " جادي عن اصل المال"

In [127]:
query_dense_vector = next(dense_embedding_model.query_embed(query_text))
query_sparse_vector = next(sparse_embedding_model.query_embed(query_text))

In [128]:
prefecth = [
    # models.Prefetch(
    #     query=query_dense_vector,
    #     using="fast-paraphrase-multilingual-minilm-l12-v2",
    #     limit=40,
    # ),
    models.Prefetch(
        query=models.SparseVector(**query_sparse_vector.as_object()),
        using="fast-sparse-bm25",
        limit=40,
    ),
]

In [129]:
result = qdrant_client.query_points(
    collection_name=collection_name,
    prefetch=prefecth,
    query=models.FusionQuery(fusion=models.Fusion.RRF),
    limit=10,
    with_payload=True,
)

In [130]:
print(result)

QueryResponse(
    points=[
        ScoredPoint(
            id=1066,
            version=33,
            score=0.5,
            payload={
                'document': 'جادي, ثمانية/ thmanyah\nحتى تعرف كيف تُصنع الفلوس\nيُشاع أن ابتكار المال جاء مباشرةً من
تبادل السلع (المقايضة)، لكن الحقيقة أنها قصة غير صحيحة، وجاءت لتسهّل شرح فكرة وصولنا إلى العملة الحديثة فقط! إذن 
السؤال: كيف يُصنع المال؟ ومن أين تأتي البنوك بالمال؟ ولماذا قروض البنوك محدودة؟ لمَ لا تطبع أموال أكثر؟ وهل تزيد 
الأموال إلى الأبد؟ - • نسعد باقتراحاتكم وتعليقاتكم على بريد البرنامج . • حمّل تطبيق « راديو ثمانية ». • مع « ودائع 
»، استثمر أموالك في الودائع الزمنية المتوافقة مع الضوابط الشرعية. الروابط: @mohammadsaadj — حساب محمد آل جابر على 
تويتر @hadi_fq — حساب هادي فقيهي على تويتر كواليس القرارات المالية في السعودية — حلقة من بودكاست سقراط مع وزير 
المالية. إغاثة الأمة بكشف الغمة — كتاب تاريخي لمصر، وجزء منه عن الغلاء الذي عاشته مصر. كيف يخلق المال — منشور من 
البنك المركزي البريطاني يشرح كيفية خلق المال. Debt — كتاب يشرح كيف وصلنا إلى العملة. The Ascent of Money — كتاب عن 
التاريخ المالي للعالم.',
                'episode_id': 216,
                'podcast_id': 51,
                'podcast_name': 'جادي',
                'podcast_author': 'ثمانية/ thmanyah',
                'podcast_categories': ['Business']
            },
            vector=None,
            shard_key=None,
            order_value=None
        ),
        ScoredPoint(
            id=1097,
            version=34,
            score=0.33333334,
            payload={
                'document': 'جادي, ثمانية/ thmanyah\nكم الراتب الذي سيجعلك سعيدًا\nدراسة غريبة تثبت أن هناك علاقة 
طردية بين ارتفاع مستوى الدخل والسعادة! وهذه الدراسة أتت لتفند العديد من الدراسات التي تقول إن سعادتك لن تزداد بعد 
دخل معين. صحيح أن السعادة مفهوم عام يصعب قياسه؛ فهو يختلف من إنسان إلى آخر، ومن ثقافة إلى أخرى، إلا أن المال من 
أساسيات الحياة، ويحسن جودة حياتك، ولكن هل فعلًا تزيد سعادتك بارتفاع دخلك؟ وهل «الفلوس» تشتري السعادة؟ نسعد 
باقتراحاتكم وتعليقاتكم على بريد البرنامج . حمّل تطبيق « راديو ثمانية » الروابط: @mohammadsaadj — حساب محمد آل جابر 
على تويتر @hadi_fq — حساب هادي فقيهي على تويتر الدخل والسعادة — دراسة تتحدث عن العلاقة الطردية بين الدخل والسعادة. 
السعادة تتوقف عند حد معين من الدخل — دراسة في 2010 تقول أن السعادة تتوقف عند دخل معين. See omnystudio.com/listener 
for privacy information.',
                'episode_id': 233,
                'podcast_id': 51,
                'podcast_name': 'جادي',
                'podcast_author': 'ثمانية/ thmanyah',
                'podcast_categories': ['Business']
            },
            vector=None,
            shard_key=None,
            order_value=None
        ),
        ScoredPoint(
            id=106,
            version=3,
            score=0.25,
            payload={
                'document': 'الفجر, ثمانية/thmanyah\nلماذا تعارك ترامب مع زيلنسكي\nقبل أيام ضجت برامج التواصل 
الاجتماعي بمقاطع لجدل حاد بين ترمب ورئيس أوكرانيا في البيت الأبيض، والذي يعد جدلًا غير اعتيادي وخصوصًا أمام الناس 
والإعلام. فلماذا ظهر هذا الجدل علنًا، وهل كان ترمب متعمد ذلك؟ كما أن خلال هذا الأسبوع أصدرت الوكالة الدولية للطاقة 
الذرية تقريرًا تحذر فيه من تسارع تخصيب اليورانيوم في إيران واقترابها من السلاح النووي. فما قصة هذا الخبر، وماذا يجري
حاليًا بين الإدارة الأمريكية وإيران، والأهم موقف السعودية أمام كل هذا. - • حمّل تطبيق « راديو ثمانية ». • تابع راديو
ثمانية على: إكس ، و إنستقرام ، و تِك توك ـــــ وراديو ثمانية. 😁 • صفا للاستثمار ، حلول سكنية راقية تلبي احتياجك 
وتسبق توقعاتك. الروابط: حتى تعرف كيف تصنع الفلوس – حلقة من بودكاست جادي تشرح كيف يصنع المال مين اللي يراقب الإنترنت
– حلقة من بودكاست رابط تشعبي تكشف العالم وراء الإنترنت اليمين يتجه لحكم العالم – حلقة سابقة نشرح فيها اليمين ونشأة 
الأحزاب Scoop — وثائقي حول مقابلة الأمير أندرو المرتبطة بفضائح إبستين قصة إيران والطاقة النووية — حلقة سابقة توضح 
قصة إيران مع الطاقة النووية السعودية واليورانيوم — حلقة سابقة عن البرنامج السعودي للطاقة النووية السلمية',
                'episode_id': 345,
                'podcast_id': 53,
        

In [131]:
for r in result.points:
    if "ثمانية" in r.payload["podcast_author"]:
        print(r.payload)

{
    'document': 'جادي, ثمانية/ thmanyah\nحتى تعرف كيف تُصنع الفلوس\nيُشاع أن ابتكار المال جاء مباشرةً من تبادل السلع
(المقايضة)، لكن الحقيقة أنها قصة غير صحيحة، وجاءت لتسهّل شرح فكرة وصولنا إلى العملة الحديثة فقط! إذن السؤال: كيف 
يُصنع المال؟ ومن أين تأتي البنوك بالمال؟ ولماذا قروض البنوك محدودة؟ لمَ لا تطبع أموال أكثر؟ وهل تزيد الأموال إلى 
الأبد؟ - • نسعد باقتراحاتكم وتعليقاتكم على بريد البرنامج . • حمّل تطبيق « راديو ثمانية ». • مع « ودائع »، استثمر 
أموالك في الودائع الزمنية المتوافقة مع الضوابط الشرعية. الروابط: @mohammadsaadj — حساب محمد آل جابر على تويتر 
@hadi_fq — حساب هادي فقيهي على تويتر كواليس القرارات المالية في السعودية — حلقة من بودكاست سقراط مع وزير المالية. 
إغاثة الأمة بكشف الغمة — كتاب تاريخي لمصر، وجزء منه عن الغلاء الذي عاشته مصر. كيف يخلق المال — منشور من البنك 
المركزي البريطاني يشرح كيفية خلق المال. Debt — كتاب يشرح كيف وصلنا إلى العملة. The Ascent of Money — كتاب عن 
التاريخ المالي للعالم.',
    'episode_id': 216,
    'podcast_id': 51,
    'podcast_name': 'جادي',
    'podcast_author': 'ثمانية/ thmanyah',
    'podcast_categories': ['Business']
}

{
    'document': 'جادي, ثمانية/ thmanyah\nكم الراتب الذي سيجعلك سعيدًا\nدراسة غريبة تثبت أن هناك علاقة طردية بين 
ارتفاع مستوى الدخل والسعادة! وهذه الدراسة أتت لتفند العديد من الدراسات التي تقول إن سعادتك لن تزداد بعد دخل معين. 
صحيح أن السعادة مفهوم عام يصعب قياسه؛ فهو يختلف من إنسان إلى آخر، ومن ثقافة إلى أخرى، إلا أن المال من أساسيات 
الحياة، ويحسن جودة حياتك، ولكن هل فعلًا تزيد سعادتك بارتفاع دخلك؟ وهل «الفلوس» تشتري السعادة؟ نسعد باقتراحاتكم 
وتعليقاتكم على بريد البرنامج . حمّل تطبيق « راديو ثمانية » الروابط: @mohammadsaadj — حساب محمد آل جابر على تويتر 
@hadi_fq — حساب هادي فقيهي على تويتر الدخل والسعادة — دراسة تتحدث عن العلاقة الطردية بين الدخل والسعادة. السعادة 
تتوقف عند حد معين من الدخل — دراسة في 2010 تقول أن السعادة تتوقف عند دخل معين. See omnystudio.com/listener for 
privacy information.',
    'episode_id': 233,
    'podcast_id': 51,
    'podcast_name': 'جادي',
    'podcast_author': 'ثمانية/ thmanyah',
    'podcast_categories': ['Business']
}

{
    'document': 'الفجر, ثمانية/thmanyah\nلماذا تعارك ترامب مع زيلنسكي\nقبل أيام ضجت برامج التواصل الاجتماعي بمقاطع 
لجدل حاد بين ترمب ورئيس أوكرانيا في البيت الأبيض، والذي يعد جدلًا غير اعتيادي وخصوصًا أمام الناس والإعلام. فلماذا ظهر
هذا الجدل علنًا، وهل كان ترمب متعمد ذلك؟ كما أن خلال هذا الأسبوع أصدرت الوكالة الدولية للطاقة الذرية تقريرًا تحذر فيه
من تسارع تخصيب اليورانيوم في إيران واقترابها من السلاح النووي. فما قصة هذا الخبر، وماذا يجري حاليًا بين الإدارة 
الأمريكية وإيران، والأهم موقف السعودية أمام كل هذا. - • حمّل تطبيق « راديو ثمانية ». • تابع راديو ثمانية على: إكس ،
و إنستقرام ، و تِك توك ـــــ وراديو ثمانية. 😁 • صفا للاستثمار ، حلول سكنية راقية تلبي احتياجك وتسبق توقعاتك. 
الروابط: حتى تعرف كيف تصنع الفلوس – حلقة من بودكاست جادي تشرح كيف يصنع المال مين اللي يراقب الإنترنت – حلقة من 
بودكاست رابط تشعبي تكشف العالم وراء الإنترنت اليمين يتجه لحكم العالم – حلقة سابقة نشرح فيها اليمين ونشأة الأحزاب 
Scoop — وثائقي حول مقابلة الأمير أندرو المرتبطة بفضائح إبستين قصة إيران والطاقة النووية — حلقة سابقة توضح قصة إيران
مع الطاقة النووية السعودية واليورانيوم — حلقة سابقة عن البرنامج السعودي للطاقة النووية السلمية',
    'episode_id': 345,
    'podcast_id': 53,
    'podcast_name': 'الفجر',
    'podcast_author': 'ثمانية/thmanyah',
    'podcast_categories': ['News', 'Daily']
}

{
    'document': 'بودكاست مربّع | مع حاتم النجار, ثمانية/thmanyah\nكيف تضاعف دخلك\nينقسم الأشخاص عند الحديث عن المال
إلى قسمين، فهناك من يعتقد بأن الطريق إلى المليون سهل ولكنه يحتاج إلى الذكاء والاجتهاد، وهناك الموظفون الذين 
يخالفونهم الرأي! ضيفنا في هذه الحلقة عماد منشي الذي ولد في عائلة ثرية، إذ كان جده من أصحاب التجارات في منطقة البلد 
في محافظة جدة. يشاركنا عماد تجاربه المختلفة في التجارة والأعمال بعدما رأى كيف تُصنع الأموال في مختلف الأعمال، ففشل 
مرات ونجح مرة. اكتشف ضيفنا بعد التجارب أن المال ليس كل شيء؛ فبدأ يبحث عن شغفه الذي وجده في الاستشارات التي جعلته 
يطلع على تجارب عديدة لمختلف التجار. شاركنا في هذه الحلقة خلاصة هذه التجارب. فما المال، وكيف نصبح أثرياء، وما 
الأخطاء التي يقع فيها رواد الأعمال، وماذا عن شراء بيت العمر؟ – • اقترح ضيفًا لبودكاست مربع بمراسلتنا على . • حمّل 
تطبيق « راديو ثمانية » • عصر جديد للصحافة مع موقع وتطبيق صحيفة الشرق الأوسط . • تملّك وحدات سكنية ذكية بخدمات عقارية
مستدامة وحلول شراء مبتكرة، مع وحدات صفا للاستثمار . • تطبيق « محلي » من سلة، أكثر من 7 ملايين منتج فريد من 47 ألف 
متجر سعودي. • « تكافل الراجحي » حلول تأمينية مبتكرة وموثوقة. • سواء أكنت من عشّاق القهوة بتحضير صنّاعها المحترفين أم
تفضّل تحضيرها في المنزل. نشْرُف بإثرائك في عالم القهوة المختصة، ابدأ بخطوة وشاركنا تجربتك عبر منصاتنا التفاعلية. 
الروابط: @emad_munshi — حساب عماد منشي على إنستقرام معنى أن تجد شغفك بعد 15 عامًا من الدراسة — حلقة محمد حكيم متى 
تغير وظيفتك — حلقة هشام رزق لماذا نفشل في تحقيق أهدافنا — حلقة عبدالرحمن باسهل لماذا يعد الانشغال الدائم خوفًا — 
حلقة أحمد مشرف كيف تعيش سعيدًا بعد التقاعد — حلقة عبدالله السعدون على بودكاست فنجان See omnystudio.com/listener for
privacy information.',
    'episode_id': 870,
    'podcast_id': 63,
    'podcast_name': 'بودكاست مربّع | مع حاتم النجار',
    'podcast_author': 'ثمانية/thmanyah',
    'podcast_categories': ['Society', 'Culture', 'Personal', 'Journals']
}

In [132]:
for r in result.points:
    print(r.payload)
    print(f"{r.score:.3}")

{
    'document': 'جادي, ثمانية/ thmanyah\nحتى تعرف كيف تُصنع الفلوس\nيُشاع أن ابتكار المال جاء مباشرةً من تبادل السلع
(المقايضة)، لكن الحقيقة أنها قصة غير صحيحة، وجاءت لتسهّل شرح فكرة وصولنا إلى العملة الحديثة فقط! إذن السؤال: كيف 
يُصنع المال؟ ومن أين تأتي البنوك بالمال؟ ولماذا قروض البنوك محدودة؟ لمَ لا تطبع أموال أكثر؟ وهل تزيد الأموال إلى 
الأبد؟ - • نسعد باقتراحاتكم وتعليقاتكم على بريد البرنامج . • حمّل تطبيق « راديو ثمانية ». • مع « ودائع »، استثمر 
أموالك في الودائع الزمنية المتوافقة مع الضوابط الشرعية. الروابط: @mohammadsaadj — حساب محمد آل جابر على تويتر 
@hadi_fq — حساب هادي فقيهي على تويتر كواليس القرارات المالية في السعودية — حلقة من بودكاست سقراط مع وزير المالية. 
إغاثة الأمة بكشف الغمة — كتاب تاريخي لمصر، وجزء منه عن الغلاء الذي عاشته مصر. كيف يخلق المال — منشور من البنك 
المركزي البريطاني يشرح كيفية خلق المال. Debt — كتاب يشرح كيف وصلنا إلى العملة. The Ascent of Money — كتاب عن 
التاريخ المالي للعالم.',
    'episode_id': 216,
    'podcast_id': 51,
    'podcast_name': 'جادي',
    'podcast_author': 'ثمانية/ thmanyah',
    'podcast_categories': ['Business']
}

0.5

{
    'document': 'جادي, ثمانية/ thmanyah\nكم الراتب الذي سيجعلك سعيدًا\nدراسة غريبة تثبت أن هناك علاقة طردية بين 
ارتفاع مستوى الدخل والسعادة! وهذه الدراسة أتت لتفند العديد من الدراسات التي تقول إن سعادتك لن تزداد بعد دخل معين. 
صحيح أن السعادة مفهوم عام يصعب قياسه؛ فهو يختلف من إنسان إلى آخر، ومن ثقافة إلى أخرى، إلا أن المال من أساسيات 
الحياة، ويحسن جودة حياتك، ولكن هل فعلًا تزيد سعادتك بارتفاع دخلك؟ وهل «الفلوس» تشتري السعادة؟ نسعد باقتراحاتكم 
وتعليقاتكم على بريد البرنامج . حمّل تطبيق « راديو ثمانية » الروابط: @mohammadsaadj — حساب محمد آل جابر على تويتر 
@hadi_fq — حساب هادي فقيهي على تويتر الدخل والسعادة — دراسة تتحدث عن العلاقة الطردية بين الدخل والسعادة. السعادة 
تتوقف عند حد معين من الدخل — دراسة في 2010 تقول أن السعادة تتوقف عند دخل معين. See omnystudio.com/listener for 
privacy information.',
    'episode_id': 233,
    'podcast_id': 51,
    'podcast_name': 'جادي',
    'podcast_author': 'ثمانية/ thmanyah',
    'podcast_categories': ['Business']
}

0.333

{
    'document': 'الفجر, ثمانية/thmanyah\nلماذا تعارك ترامب مع زيلنسكي\nقبل أيام ضجت برامج التواصل الاجتماعي بمقاطع 
لجدل حاد بين ترمب ورئيس أوكرانيا في البيت الأبيض، والذي يعد جدلًا غير اعتيادي وخصوصًا أمام الناس والإعلام. فلماذا ظهر
هذا الجدل علنًا، وهل كان ترمب متعمد ذلك؟ كما أن خلال هذا الأسبوع أصدرت الوكالة الدولية للطاقة الذرية تقريرًا تحذر فيه
من تسارع تخصيب اليورانيوم في إيران واقترابها من السلاح النووي. فما قصة هذا الخبر، وماذا يجري حاليًا بين الإدارة 
الأمريكية وإيران، والأهم موقف السعودية أمام كل هذا. - • حمّل تطبيق « راديو ثمانية ». • تابع راديو ثمانية على: إكس ،
و إنستقرام ، و تِك توك ـــــ وراديو ثمانية. 😁 • صفا للاستثمار ، حلول سكنية راقية تلبي احتياجك وتسبق توقعاتك. 
الروابط: حتى تعرف كيف تصنع الفلوس – حلقة من بودكاست جادي تشرح كيف يصنع المال مين اللي يراقب الإنترنت – حلقة من 
بودكاست رابط تشعبي تكشف العالم وراء الإنترنت اليمين يتجه لحكم العالم – حلقة سابقة نشرح فيها اليمين ونشأة الأحزاب 
Scoop — وثائقي حول مقابلة الأمير أندرو المرتبطة بفضائح إبستين قصة إيران والطاقة النووية — حلقة سابقة توضح قصة إيران
مع الطاقة النووية السعودية واليورانيوم — حلقة سابقة عن البرنامج السعودي للطاقة النووية السلمية',
    'episode_id': 345,
    'podcast_id': 53,
    'podcast_name': 'الفجر',
    'podcast_author': 'ثمانية/thmanyah',
    'podcast_categories': ['News', 'Daily']
}

...

0.111

{
    'document': 'بودكاست مربّع | مع حاتم النجار, ثمانية/thmanyah\nكيف تضاعف دخلك\nينقسم الأشخاص عند الحديث عن المال
إلى قسمين، فهناك من يعتقد بأن الطريق إلى المليون سهل ولكنه يحتاج إلى الذكاء والاجتهاد، وهناك الموظفون الذين 
يخالفونهم الرأي! ضيفنا في هذه الحلقة عماد منشي الذي ولد في عائلة ثرية، إذ كان جده من أصحاب التجارات في منطقة البلد 
في محافظة جدة. يشاركنا عماد تجاربه المختلفة في التجارة والأعمال بعدما رأى كيف تُصنع الأموال في مختلف الأعمال، ففشل 
مرات ونجح مرة. اكتشف ضيفنا بعد التجارب أن المال ليس كل شيء؛ فبدأ يبحث عن شغفه الذي وجده في الاستشارات التي جعلته 
يطلع على تجارب عديدة لمختلف التجار. شاركنا في هذه الحلقة خلاصة هذه التجارب. فما المال، وكيف نصبح أثرياء، وما 
الأخطاء التي يقع فيها رواد الأعمال، وماذا عن شراء بيت العمر؟ – • اقترح ضيفًا لبودكاست مربع بمراسلتنا على . • حمّل 
تطبيق « راديو ثمانية » • عصر جديد للصحافة مع موقع وتطبيق صحيفة الشرق الأوسط . • تملّك وحدات سكنية ذكية بخدمات عقارية
مستدامة وحلول شراء مبتكرة، مع وحدات صفا للاستثمار . • تطبيق « محلي » من سلة، أكثر من 7 ملايين منتج فريد من 47 ألف 
متجر سعودي. • « تكافل الراجحي » حلول تأمينية مبتكرة وموثوقة. • سواء أكنت من عشّاق القهوة بتحضير صنّاعها المحترفين أم
تفضّل تحضيرها في المنزل. نشْرُف بإثرائك في عالم القهوة المختصة، ابدأ بخطوة وشاركنا تجربتك عبر منصاتنا التفاعلية. 
الروابط: @emad_munshi — حساب عماد منشي على إنستقرام معنى أن تجد شغفك بعد 15 عامًا من الدراسة — حلقة محمد حكيم متى 
تغير وظيفتك — حلقة هشام رزق لماذا نفشل في تحقيق أهدافنا — حلقة عبدالرحمن باسهل لماذا يعد الانشغال الدائم خوفًا — 
حلقة أحمد مشرف كيف تعيش سعيدًا بعد التقاعد — حلقة عبدالله السعدون على بودكاست فنجان See omnystudio.com/listener for
privacy information.',
    'episode_id': 870,
    'podcast_id': 63,
    'podcast_name': 'بودكاست مربّع | مع حاتم النجار',
    'podcast_author': 'ثمانية/thmanyah',
    'podcast_categories': ['Society', 'Culture', 'Personal', 'Journals']
}

0.1

{
    'document': 'كنبة السبت, Mics | مايكس\nالمال: فكرة\nنتحدث في هذه الحلقة من بودكاست #كنبة_السبت عن تصحيح بعض 
المفاهيم التي تساعدنا في تجنب التبذير والاستثمار في عقليتنا، وفهم المال بطريقة سليمة وترتيب أفكارنا تجاهه، مع أهمية
استشارة المختصين الماليين بآليات وطرق الوصول إلى الثراء. تأتيكم هذه الحلقة برعاية - مركز التواصل والمعرفة المالية 
الموقع الالكتروني: اكس: اليوتيوب: لينكدإن: تيك توك: الحلقة 89 من بودكاست «كنبة السبت» مع د. أفنان الغامدي. بإمكانك 
مشاهدة الحلقة من خلال اليوتيوب أو الاستماع لها عبر منصات البودكاست. وكذلك يهمنا معرفة رأيك عن الحلقات بالتعليقات أو
تقييمك على Apple Podcast. كما بوسعك تقديم اقتراحك لبودكاست كنبة السبت بمراسلتنا على: اشترك الآن في كنبة+ لتتعرف على
نفسك بشكل أعمق، مع أكثر من سلسلة حلقات حصرية. الآن نطرح (سلسلة سؤال وجواب) حيث ستسمع د. أفنان سؤالك وتجيب على 
تساؤلاتكم المشتركة التي تلامس شريحة واسعة.. سجل سؤالك هنا: اطلب الآن من متجر كنبة السبت: حساب د. أفنان الغامدي في 
تويتر: لمعرفة أحدث أخبار شبكة مايكس تابعنا على: تويتر: انستقرام: فيس بوك: لينكد ان: تيك توك:',
    'episode_id': 3807,
    'podcast_id': 97,
    'podcast_name': 'كنبة السبت',
    'podcast_author': 'Mics | مايكس',
    'podcast_categories': ['Society', 'Culture']
}

0.0909